In [13]:
import numpy as np
import glob
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from scipy.stats import wilcoxon

In [14]:
fileNames2D = np.sort(glob.glob("./data/zheng/processed_interactions/*"))[:5]

In [15]:
def get_probabilities(dataframe,current_phase):
    map = ['same','modify-x','modify-y','modify-z','modify-x-y','modify-y-z','modify-x-z','modify-x-y-z']
    dict_map = dict.fromkeys(map, 0)
    for action in dataframe['Action']:
        dict_map[action] += 1
    return dict_map[current_phase] / sum(dict_map.values())


In [16]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None
        self.trend= None

    def Wilcoxon_Test(self,series_X, series_Y):

        return wilcoxon(series_X, series_Y)


In [18]:

all_pvalue=[]
for state in['same','modify-x','modify-y','modify-z','modify-x-y','modify-y-z','modify-x-z','modify-x-y-z']:
    print(f"\nProcessing test for state: {state}")
     # Create an empty DataFrame to store test data
    test_dfs = pd.DataFrame(columns=['User', 'First_Half', 'Second_Half'])
    # Iterate through file names
    for i, u in enumerate(fileNames2D):
        # Read CSV file
        df = pd.read_csv(u)

        # Extract user information from file path


        #print(f"\nProcessing data for user: {u}")

        # Split the DataFrame into two halves
        mid = round(len(df) / 2)
        slice1 = df[:mid].reset_index(drop=True)
        slice2 = df[mid:].reset_index(drop=True)
        #print('length of slice',len(slice1), len(slice2))

        # Calculate Navigation probabilities for each half
        probab_1 = get_probabilities(slice1,state)
        probab_2 = get_probabilities(slice2,state)

        #print(f"Probabilities for {u} - First Half: {probab_1}, Second Half: {probab_2}")

        # Create a DataFrame with user, first_half, and second_half probabilities
        user_df = pd.DataFrame({'User': [u], 'First_Half': [probab_1], 'Second_Half': [probab_2]})

        # Concatenate the result to the test_dfs DataFrame for all users
        test_dfs = pd.concat([test_dfs, user_df], ignore_index=True)

    # Perform Wilcoxon signed-rank test on the probabilities of the two halves
    stats_test = StationarityTests()
    result = stats_test.Wilcoxon_Test(test_dfs['First_Half'], test_dfs['Second_Half'])
    #print(result)
    del test_dfs

    print(f"State: {state} , Probabilities: {probab_1} {probab_2} , Users are Non-stationary {result.pvalue<0.05} with p-value {result.pvalue}")
    all_pvalue.append(result.pvalue)



Processing test for state: same
State: same , Probabilities: 0.6172839506172839 0.6049382716049383 , Users are Non-stationary False with p-value 0.7150006546880893

Processing test for state: modify-x
State: modify-x , Probabilities: 0.037037037037037035 0.0 , Users are Non-stationary False with p-value 0.8125

Processing test for state: modify-y
State: modify-y , Probabilities: 0.012345679012345678 0.012345679012345678 , Users are Non-stationary False with p-value 0.4652088184521418

Processing test for state: modify-z
State: modify-z , Probabilities: 0.25925925925925924 0.2716049382716049 , Users are Non-stationary False with p-value 0.5929800980174267

Processing test for state: modify-x-y
State: modify-x-y , Probabilities: 0.024691358024691357 0.024691358024691357 , Users are Non-stationary False with p-value 0.7150006546880893

Processing test for state: modify-y-z
State: modify-y-z , Probabilities: 0.024691358024691357 0.06172839506172839 , Users are Non-stationary False with p-

/var/folders/w6/jn7_623d6d96165_8x9l_vqh0000gn/T/ipykernel_11882/1151033281.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test_dfs = pd.concat([test_dfs, user_df], ignore_index=True)
/Users/aryal/anaconda3/envs/ForeCache-Master/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/aryal/anaconda3/envs/ForeCache-Master/lib/python3.9/site-packages/scipy/stats/_morestats.py:4102: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/var/folders/w6/jn7_623d6d96165_8x9l_vqh00

In [6]:
from scipy import stats
stats.false_discovery_control(all_pvalue,method='bh')

array([0.50192072, 0.55888545, 0.50072108, 0.50072108, 0.50072108,
       0.50072108, 0.41618218, 0.93591497])

In [112]:
from statsmodels.stats.multitest import multipletests
multipletests(all_pvalue,alpha=0.05,method='bonferroni')

(array([False,  True, False]),
 array([1.        , 0.00813675, 0.07185173]),
 0.016952427508441503,
 0.016666666666666666)

In [8]:
test_dfs

NameError: name 'test_dfs' is not defined